In [1]:
import numpy as np
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib
from pyscf.cc import CCSD
from pyscf.data import nist
import matplotlib.pyplot as plt
import basis_set_exchange as bse
angstrom = 1 / 0.52917721067

In [56]:
COmol = pyscf.gto.Mole(atom = 'C 0 0 0.; O 0 0 2.',basis='6-31G',unit='au')
COmol.build()
COHF = pyscf.scf.RHF(COmol)
NNmol = pyscf.gto.Mole(atom = 'N 0 0 0.; N 0 0 2.',basis='6-31G',unit='au')
NNmol.build()
NNHF = pyscf.scf.RHF(NNmol)

In [57]:
NNHF.scf()
nnccsd=CCSD(NNHF)

converged SCF energy = -108.864599611748


In [4]:
def Alch_calc(calc,deltaZ):
    mf=pyscf.qmmm.mm_charge(calc,calc.mol.atom_coords()/angstrom,deltaZ)
    includeonly = [0, 1]
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):            
            q = mf.mol.atom_charges().astype(np.float)
            q[includeonly] -= deltaZ
            return mf.mol.energy_nuc(q)
    mf = NoSelfQMMM()
    return mf

In [9]:
def Alch_e_rdm1(calc,deltaZ):
    mf=Alch_calc(calc,deltaZ)
    mf.scf()
    mycc=CCSD(mf)
    mycc.run()
    e=mycc.e_tot
    dm1=mycc.make_rdm1()
    dm1_ao = np.einsum('pi,ij,qj->pq', mf.mo_coeff, dm1, mf.mo_coeff.conj())
    return [e,dm1_ao]

In [10]:
COdm=Alch_e_rdm1(COHF,np.array((0.,0.)))[1]

converged SCF energy = -112.652239281562
E(CCSD) = -112.8512054563891  E_corr = -0.1989661748272975


In [46]:
"""returns the delta V matrix in ao basis for the alchemical perturbation"""
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1]/angstrom)
    dV+=mol.intor('int1e_rinv')*dL[1]
    return dV

In [12]:
# Target molecule NN@CO 
Alch_calc(COHF,np.array((-1.,1.))).kernel()-COHF.kernel()

converged SCF energy = -105.911201761195
converged SCF energy = -112.652239281562


6.741037520366575

In [13]:
np.einsum('ij,ji',COdm,DeltaV(COmol,np.array((1.,-1.))))

6.758646049736087

In [36]:
e_dm=[]
for i in np.linspace(-.2,.2,5):
    e_dm.append(Alch_e_rdm1(COHF,np.array((-i,i))))

converged SCF energy = -111.144854160313
E(CCSD) = -111.3380738007273  E_corr = -0.1932196404147349
converged SCF energy = -114.233704023131
E(CCSD) = -114.4384906811646  E_corr = -0.2047866580339281
converged SCF energy = -113.433786714567
E(CCSD) = -113.6356613346309  E_corr = -0.2018746200637949
converged SCF energy = -112.652239281562
E(CCSD) = -112.8512054548212  E_corr = -0.1989661732594031
converged SCF energy = -111.889199239812
E(CCSD) = -112.0852751270358  E_corr = -0.1960758872234131
converged SCF energy = -111.144854160313
E(CCSD) = -111.3380738005284  E_corr = -0.1932196402157064


In [49]:
es=np.asarray([e_dm[x][0] for x in range(len(e_dm))])
dms=np.asarray([e_dm[x][1] for x in range(len(e_dm))])
dms_array=np.asarray(dms)
e_CO=es[2]
e_NN_at_CO=Alch_e_rdm1(COHF,np.array((-1,1)))[0]

converged SCF energy = -105.911201761195
E(CCSD) = -106.0859430665793  E_corr = -0.1747413053840686


In [20]:
coeff5p=np.asarray([[1/12,-2/3,0,2/3,-1/12],[-1/12,4/3,-5/2,4/3,-1/12],[-1/2,1,0,-1,1/2],[1.,-4.,6.,-4.,1.]]) #f',f",f^3,f^4
def tayl_exp_5p(pts,dh,dL):
    pred=pts[2]
    for i in range(4):
        pred+=np.sum(pts*coeff5p[i])/np.math.factorial(i+1)*(dL/dh)**(i+1)
    return (pred)

def rho_tilda(ptsIN,dh,dL):
    pts=ptsIN.copy()
    pred=pts[2]
    for i in range(4):
        deriv=np.zeros_like(pred)
        for j in range(5):
            deriv+=(pts[j]*coeff5p[i][j])
        pred+=deriv/np.math.factorial(i+2)*(dL/dh)**(i)
    return (pred)

In [21]:
pts=dms
deriv=np.zeros_like(pts[2])
for j in range(5):
    deriv+=(pts[j]*coeff5p[3][j])
pts[1].shape

(18, 18)

In [54]:
dV_CO=DeltaV(COmol,np.array((1.,-1.)))

In [55]:
#CO->NN 
dVNN=1/2
dE1=np.einsum('ij,ji',dms[2],dV_CO)
print ('first order alchemy correction = ',dVNN+dE1)
dRho1=(dms[3]-dms[1])/2/.1
dE2=np.einsum('ij,ji',dRho1,dV_CO)/2
print ('second order alchemy correction = ',dVNN+dE1+dE2)
dRho2=(dms[3]-2*dms[2]+dms[1])/.01
dE3=np.einsum('ij,ji',dRho2,dV_CO)/6
print ('third order alchemy correction = ', dVNN+dE1+dE2+dE3)
dRho3=(dms[4]*0.5-dms[3]+dms[1]-dms[0]*0.5)/.001
dE4=np.einsum('ij,ji',dRho3,dV_CO)/24
print ('fourth order alchemy correction = ', dVNN+dE1+dE2+dE3+dE4)
dRho4=(dms[4]-4*dms[3]+6*dms[2]-4*dms[1]+dms[0])/.0001
dE5=np.einsum('ij,ji',dRho4,dV_CO)/120
print ('fifth order alchemy correction = ', dVNN+dE1+dE2+dE3+dE4+dE5)
print('Target = ',e_NN_at_CO-e_CO)
print ('\nError due to alchemy = ', e_NN_at_CO-e_CO-(dVNN+dE1+dE2+dE3+dE4+dE5) )
print ('Error due to basis set = ', e_NN_at_CO-e_CO )
print(dE1,dE2,dE3,dE4,dE5)

first order alchemy correction =  7.258646046157674
second order alchemy correction =  6.83232074504474
third order alchemy correction =  6.8023687828593635
fourth order alchemy correction =  6.781077720240949
fifth order alchemy correction =  6.773545410196495
Target =  6.765262388241851

Error due to alchemy =  -0.008283021954643921
6.758646046157674 -0.426325301112934 -0.029951962185376168 -0.02129106261841431 -0.007532310044454283
